# Computation of repartition keys, valuation weights and dedicated values

### Necessary imports

In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import pandas as pd
import numpy as np
from pathlib import Path
from TPT_generator_python import Data_Bucket, TPT_Fetcher

### Variables definition

In [2]:
DATE = pd.to_datetime("2020-12-31").date()
CLIENT = "Dynasty"
ISIN = "LU1280365476"
SOURCE_DIR = Path("../data")

In [3]:
#fetcher = TPT_Fetcher(DATE, CLIENT, ISIN, SOURCE_DIR)
bucket = Data_Bucket(DATE, CLIENT, ISIN, SOURCE_DIR)

### Acquiring datas from database
#### Instruments infos
Using a Data_Bucket object which abstract database accesses through the fetcher object (see documentation), we get all necessary data from the database and store them in a dataframe.

We get the list of instruments in the subfund's portfolio, their market values and the dedication indicator which define the group of shareclasses, the instrument must be distributed to.

In [4]:
instruments = bucket.get_instruments(indicator="all", info=["hedge_indicator",
                                                            "market_and_accrued_fund"])
instruments = instruments.sort_index()
instruments.head()

,hedge_indicator,market_and_accrued_fund
14_Identification code of the financial instrument,,
BE6286986284,LU6517-NH,6501000.00
CA01CHF,LU6517-NH,206.58
CA01CHFHA,LU6517-HA-C2-C5-D1,4355919.83
CA01EUR,LU6517-NH,25423090.51
CA01EURHA,LU6517-HA-C2-C5-D1,17557.09


#### Shareclasses infos
Here we get the list of shareclasses in the subfund, then acquire their NAV and list of groups they're included into.

In [5]:
shareclasses = bucket.get_subfund_shareclasses()
print(shareclasses)

['LU1280365633', 'LU1280365476', 'LU1280365393', 'LU1280365559', 'LU1483663818', 'LU1840818220', 'LU1586705938', 'LU1508332993']


In [6]:
NAVs = pd.DataFrame(index=shareclasses, 
                    columns=["shareclass_total_net_asset_sf_curr",
                             "subfund_total_net_asset",
                             "indicators"],
                    dtype=object)

NAVs["indicators"] = NAVs["indicators"].astype(object)
for isin in shareclasses:
    NAVs.loc[isin, "shareclass_total_net_asset_sf_curr"] = \
        bucket.get_shareclass_nav(isin=isin, info="shareclass_total_net_asset_sf_curr")
    NAVs.loc[isin, "subfund_total_net_asset"] = \
        bucket.get_shareclass_nav(isin=isin, info="subfund_total_net_asset")
    NAVs.at[isin, "indicators"] = \
        [bucket.get_subfund_infos("subfund_indicator"),
         bucket.get_shareclass_infos(isin=isin, info="shareclass"),
         bucket.get_shareclass_infos(isin=isin, info="shareclass_id")
                                   ]
NAVs

,shareclass_total_net_asset_sf_curr,subfund_total_net_asset,indicators
LU1280365633,1.33117e+08,3.30155e+08,"[LU6517-NH, B, LU6517-NH]"
LU1280365476,NaN,3.30155e+08,"[LU6517-NH, A, LU6517-HA-C2-C5-D1]"
LU1280365393,1.0327e+08,3.30155e+08,"[LU6517-NH, A, LU6517-NH]"
LU1280365559,NaN,3.30155e+08,"[LU6517-NH, A, LU6517-HB-C3-C7]"
LU1483663818,NaN,3.30155e+08,"[LU6517-NH, B, LU6517-HA-C2-C5-D1]"
LU1840818220,NaN,3.30155e+08,"[LU6517-NH, B, LU6517-HC-C8]"
LU1586705938,NaN,3.30155e+08,"[LU6517-NH, B, LU6517-HB-C3-C7]"
LU1508332993,1.18921e+07,3.30155e+08,"[LU6517-NH, D, LU6517-NH]"


## Computation of the distributions matrices
To compute the repartition keys, valuation weights and dedicated values for each instruments and shareclasses in a subfund, we proceed by computing a set of distributions matrices at the subfund level. This approache allows us to perform more checks and assert the coherence of our calculations at the subfund level. 

###  Indicator matrix (BETAS)
The first distribution matrice we will use is the BETAS, it is a binary matrice built by doing the cartesian product of the shareclasses with the instruments.

Each $\beta_{i,j} = BETAS(i,j)$ is defined as follows:
$$
\beta_{i,j} = \left\{
    \begin{array}\\
        1 & \mbox{if } \ \text{instrument i is distributed to shareclass j}\\
        0 & \mbox{if } \ \text{not} \\
    \end{array}
\right.
$$

We then append a column "fund" to the matrix BETAS which is the product of all column for each row.

In [7]:
BETAS = pd.DataFrame(1, index=instruments.index, columns=NAVs.index)
for isin in shareclasses:
    BETAS[isin].where(
        instruments["hedge_indicator"].isin(NAVs.loc[isin, "indicators"]),
        0,
        inplace=True)
BETAS.sort_index(inplace=True)
BETAS["fund"] = 1
for isin in shareclasses:
    BETAS["fund"] = BETAS["fund"] * BETAS[isin]

BETAS.head()

,LU1280365633,LU1280365476,LU1280365393,LU1280365559,LU1483663818,LU1840818220,LU1586705938,LU1508332993,fund
14_Identification code of the financial instrument,,,,,,,,,
BE6286986284,1,1,1,1,1,1,1,1,1
CA01CHF,1,1,1,1,1,1,1,1,1
CA01CHFHA,0,1,0,0,1,0,0,0,0
CA01EUR,1,1,1,1,1,1,1,1,1
CA01EURHA,0,1,0,0,1,0,0,0,0


### Shareclasses Key matrix (SK)
The second distribution matrice we will use is the Shareclasses Key, it is built in the same way as the BETAS matrix (excluding the added colun "fund"). Where each SK(i,j) is defined as follow:
$$
SK(i,j) = \left\{
    \begin{array}\\
        \text{NAV of shareclass j} & \mbox{if } \ \beta_{i,j} = 1 \\
        0 & \mbox{if } \ \beta_{i,j} = 0 \\
    \end{array}
\right.
$$
the NAV of the shareclass j if instrument j is distributed to j. 

In [8]:
SK = pd.DataFrame(0, index=instruments.index, columns=NAVs.index)
for isin in shareclasses:
    SK[isin].where(
        ~(instruments["hedge_indicator"].isin(NAVs.loc[isin, "indicators"])),
        NAVs.loc[isin, "shareclass_total_net_asset_sf_curr"].astype('float64'),
        inplace=True)
SK.sort_index(inplace=True)
SK.head()

AttributeError: 'float' object has no attribute 'astype'

### Dedicated values matrix (D)
The third matrice we will use is the dedicated value matrice, where each $d_{i,j}$ is defined as the value of instrument i associated to the shareclass j.

In the general case, the value of instrument i is distributed over each shareclass it is associated with proportionately to the respective NAV of the shareclasses.

The general definition of $d_{i,j}$ is as follow:
$$
d_{i,j} = I_i \cdot \frac{\beta_{i,j} S_j}{\sum_{k=0}^{m} \beta_{i,k} S_k}
$$

#### Repartition keys matrix (ALPHAS)

We can then define the repartition key $\alpha_{i,j}$ of each instrument i to each shareclass j such that:

$$
d_{i,j} = \alpha_{i,j} \cdot I_i
$$

which gives:

$$
\alpha_{i,j} = \frac{\beta_{i,j} S_j}{\sum_{k=0}^{m} \beta_{i,k} S_k}
$$

In [ ]:
ALPHAS = pd.DataFrame(0, index=instruments.index, columns=NAVs.index)
for isin in shareclasses:
    ALPHAS[isin] = SK[isin] * 1 / SK.sum(axis=1)
ALPHAS.sort_index(inplace=True)
ALPHAS.head()

#### Computing D (first version)
Once we have the ALPHAS matrix we should be able to compute the value $d_{i,j}$ of each instrument i dedicated to each shareclass j by multiplying each instrument value $I_i$ by $\alpha_{i,j}$.

In [ ]:
D = pd.DataFrame(0, index=instruments.index, columns=NAVs.index)
for isin in shareclasses:
    D[isin].where(
        ~(instruments["hedge_indicator"].isin(NAVs.loc[isin, "indicators"])),
        instruments["market_and_accrued_fund"]*ALPHAS[isin],
        inplace=True)
D.sort_index(inplace=True)
D.head()

#### Checking the results
To check the results, we simply have to sum the dedicated values $d_{i,j}$ for each shareclass j and we should find the NAV of each shareclass.

In [ ]:
for isin in shareclasses:
    print(f"NAV {isin} :", NAVs.loc[isin, "shareclass_total_net_asset_sf_curr"])
    print("computed NAV: ", D[isin].sum(), "\n")

#### Erroneous values
As we can see, the computed NAVs does not match the NAVs given by the clients, which means that we are doing something wrong.

###  Investigation
Let's investigate our datas to find where is our error.

#### Check that we have all the instruments
First, we begin by checking that the values of all instruments and the NAVs of all shareclasses both sum to the total NAV of the subfund.

In [ ]:
print("total NAV of the subfund: ", NAVs["subfund_total_net_asset"].iloc[0])
print("sum of shareclasses' NAV: ", NAVs["shareclass_total_net_asset_sf_curr"].sum())
print("sum of instruments values: ", instruments["market_and_accrued_fund"].sum())

As we can see, there is no mismatch in the datas we are given, this means that the errors comes from our computations.

#### Check for value loss

Now let's check that we are not ommitting any amount and that we do not lose (or create) any money along the way. In order to do that we will check that the sum of distributed values for each instrument sum to 100% of the instrument value.

In [ ]:
percentage = D.sum(axis=1) / instruments["market_and_accrued_fund"]
print(percentage.head())
print("erroneous instruments: ", percentage.loc[percentage.round(5)!=1])

All distributions indeed sum to 100% meaning that we do not lose any value during the distribution. Rather, this means that we do not correctly distribute our instruments over the shareclasses.

#### Computing D (correct version)
In fact the discrepencies between our computed NAVs and the NAVs provided by the client comes from our definition of our $d_{i,j}$ values. 

To correctly distribute the values of the instruments over the shareclasses we have to split our instruments into two categories: 
- the instruments which are at the subfund's level and must be distributed over all shareclasses of the subfund
- the instruments which are dedicated to a strict sub-set of shareclasses in the subfund.

let's denote :
- $\mathcal{I}_s$ the set of instruments dedicated to the whole subfund
- $\mathcal{I}_d$ the set of instruments dedicated to a strict subset of shareclasses

For $I_i \in \mathcal{I}_d$ we keep the same definition of $d_{i,j}$:

$$
d_{i,j} = I_i \cdot \frac{\beta_{i,j} S_j}{\sum_{k=0}^{m} \beta_{i,k} S_k}
$$

However for $I_i \in \mathcal{I}_s$ we exclude the values of all instruments $I_k \in \mathcal{I}_d$ which leads to:

$$
d_{i,j} = I_i \cdot \frac{S_j - (\beta_{i,j} \cdot \sum_{k | I_k \in \mathcal{I}_d} d_{k,j})}
                         {\sum_{k | I_k \in \mathcal{I}_s} S_k - \sum_{k | I_k \in \mathcal{I}_d} I_k} 
$$

In [ ]:
D = pd.DataFrame(0, index=instruments.index, columns=NAVs.index)
for isin in shareclasses:
    D.loc[BETAS["fund"]==0, isin] = \
        instruments.loc[BETAS["fund"]==0, "market_and_accrued_fund"] \
        * SK.loc[BETAS["fund"]==0, isin] / SK.loc[BETAS["fund"]==0].sum(axis=1)

for isin in shareclasses:
    D.loc[BETAS["fund"]==1, isin] = \
        instruments.loc[BETAS["fund"]==1, "market_and_accrued_fund"] \
        * (SK.loc[BETAS["fund"]==1, isin] \
           - D.loc[((BETAS[isin]==1) & (BETAS["fund"]==0)), isin].sum()) \
        / (SK.loc[BETAS["fund"]==1].sum(axis=1) \
           - instruments.loc[BETAS["fund"]==0, "market_and_accrued_fund"].sum())

D.sort_index(inplace=True)
D.head()

#### Checking the results
To check the results, we simply have to sum the dedicated values $d_{i,j}$ for each shareclass j and we should find the NAV of each shareclass.

In [ ]:
for isin in shareclasses:
    print(f"NAV {isin} :", NAVs.loc[isin, "shareclass_total_net_asset_sf_curr"])
    print("computed NAV: ", D[isin].sum(), "\n")

As we can see, we are able to retrieve the NAVs of each shareclass by summing the distributed values of all instruments.

In [ ]:
Valuation_weight = D.apply(lambda x: x/NAVs.loc[x.name, "shareclass_total_net_asset_sf_curr"]) 
#pd.DataFrame(index=D.index, columns=D.columns)
Valuation_weight["LU1280365476"]
